In [429]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns

In [430]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

# EDA

In [431]:
df1 = pd.read_csv('data/project_data/combined_train_data.csv')
df1.shape

(173977, 5)

### Routine EDA

In [432]:
df1.head(2)

,Unnamed: 0,0,1,2,3
0,0,2016-10-30 20:51:45+00:00,DentonPolice,"Investigating mismatch of name / photo. Until then, view the source directly from ""City Jail Custody Report"" page: http://dpdjailview.cityofdenton.com/",NaN
1,1,2016-10-30 16:59:03+00:00,DentonPolice,"10/30/2016 08:44:23 Regina, 39 yrs old Bond: $10,000 DRIVING WHILE INTOXICATED 3RD OR MORE https://www.change.org/petitions/denton-police-department-make-mug-shots-available-on-request-instead-of-putting-all-of-them-online",NaN


In [433]:
df1.isna().sum()

Unnamed: 0    0     
0             0     
1             0     
2             1052  
3             120762
dtype: int64

In [434]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173977 entries, 0 to 173976
Data columns (total 5 columns):
Unnamed: 0    173977 non-null int64
0             173977 non-null object
1             173977 non-null object
2             172925 non-null object
3             53215 non-null object
dtypes: int64(1), object(4)
memory usage: 6.6+ MB


In [435]:
df1.rename(columns={'1':'user', '3':'hashtage', '2':'tweet', '0': 'dates&time'}, inplace =True) 

In [436]:
df1.head(2)

,Unnamed: 0,dates&time,user,tweet,hashtage
0,0,2016-10-30 20:51:45+00:00,DentonPolice,"Investigating mismatch of name / photo. Until then, view the source directly from ""City Jail Custody Report"" page: http://dpdjailview.cityofdenton.com/",NaN
1,1,2016-10-30 16:59:03+00:00,DentonPolice,"10/30/2016 08:44:23 Regina, 39 yrs old Bond: $10,000 DRIVING WHILE INTOXICATED 3RD OR MORE https://www.change.org/petitions/denton-police-department-make-mug-shots-available-on-request-instead-of-putting-all-of-them-online",NaN


In [437]:
df1.drop(['Unnamed: 0', 'hashtage'], axis = 1, inplace = True)

In [438]:
df1.shape

(173977, 3)

In [439]:
df1['tweet'] = df1['tweet'].str.replace('http\S+|www.\S+', '', case = False)

In [440]:
df1.head(2)

,dates&time,user,tweet
0,2016-10-30 20:51:45+00:00,DentonPolice,"Investigating mismatch of name / photo. Until then, view the source directly from ""City Jail Custody Report"" page:"
1,2016-10-30 16:59:03+00:00,DentonPolice,"10/30/2016 08:44:23 Regina, 39 yrs old Bond: $10,000 DRIVING WHILE INTOXICATED 3RD OR MORE"


In [441]:
df1['tweet'] = df1['tweet'].replace('[^a-zA-Z ]', '', regex=True)

In [442]:
df1.head()

,dates&time,user,tweet
0,2016-10-30 20:51:45+00:00,DentonPolice,Investigating mismatch of name photo Until then view the source directly from City Jail Custody Report page
1,2016-10-30 16:59:03+00:00,DentonPolice,Regina yrs old Bond DRIVING WHILE INTOXICATED RD OR MORE
2,2016-10-30 13:52:19+00:00,DentonPolice,Steven yrs old Bond SPEEDING MPH IN A MPH ZONE DL DRIVING WHILE LICENSE INVALI
3,2016-10-26 21:45:57+00:00,DentonPolice,Monique yrs old Bond POSS MARIJ lt OZ
4,2016-10-26 21:09:45+00:00,DentonPolice,Katelyn yrs old Bond PUBLIC INTOXICATIONNONALCOHOL


### Apply Filters 

In [443]:
#CR: mask idea goes to GA alumn team Nick Cote, Fausto De La Rosa Mañón and Ilya Kvyatkovskiy

# First filter layer 

# Depends on the data, filter contents can be adjusted.

mask1_1 = df1['tweet'].str.contains('roa', regex = False, case = False, na=False)
mask1_2 = df1['tweet'].str.contains('stree', regex = False, case = False, na=False)
mask1_3 = df1['tweet'].str.contains('highway', regex = False, case = False, na=False)
mask1_4 = df1['tweet'].str.contains('ramp', regex=False, case=False, na=False)
mask1_5 = df1['tweet'].str.contains('st', regex=False, case=False, na=False)
mask1_6 = df1['tweet'].str.contains('ave', regex=False, case=False, na=False)
mask1_7 = df1['tweet'].str.contains('lane', regex=False, case=False, na=False)
mask1_8 = df1['tweet'].str.contains('ln', regex=False, case=False, na=False)
mask1_9 = df1['tweet'].str.contains('drive', regex=False, case=False, na=False)
mask1_10 = df1['tweet'].str.contains('closed', regex=False, case=False,na=False)
mask1_11 = df1['tweet'].str.contains('blvd', regex=False, case=False,na=False)
mask1_12 = df1['tweet'].str.contains('boulevard', regex=False, case=False, na=False)
mask1_13 = df1['tweet'].str.contains('pkwy', regex=False, case=False, na=False)
mask1_14 = df1['tweet'].str.contains('parkway', regex=False, case=False, na=False)


layer_1 = (mask1_1 | mask1_2 | mask1_3 | mask1_4 | mask1_5
           | mask1_6 | mask1_7 | mask1_8 | mask1_9 |mask1_11
          |mask1_12 | mask1_13 | mask1_14)&(mask1_10)

In [444]:
df_1 = df1[layer_1]

In [445]:
#df_1.duplicated().sum()

In [446]:
#df_1.drop_duplicates(keep='first', inplace = True)

In [448]:
df_1.head(100)

,dates&time,user,tweet
4959,2020-02-06 22:03:12+00:00,DallasPD,DallasPD and dfrincidents are currently on location in the block of Main St regarding a suspicious package Roads are closed for safety precation at HarwoodMain St PearlCommerce PearlMain ChiefHallDPD
5684,2019-10-23 18:36:51+00:00,DallasPD,Southbound Marsh Lane at Merrell Rd is currently closed due to a high centered wheeler We will update the closure when reopened
5712,2019-10-20 17:37:14+00:00,DallasPD,Gate and at the StateFairOfTX are full and closed for parking BigTex
5718,2019-10-19 01:32:05+00:00,DallasPD,All entry gates to the StateFairOfTX are closed BigTex
5719,2019-10-19 01:18:25+00:00,DallasPD,All parking gates are full and closed at the StateFairOfTX BigTex
5733,2019-10-16 19:24:26+00:00,DallasPD,Gate at the StateFairOfTX is full and closed for parking Gates and are open
5740,2019-10-14 21:18:21+00:00,DallasPD,Gat Valet parking is closed for parking StateFairOfTX BigTex
5744,2019-10-14 18:18:02+00:00,DallasPD,TrafficAlert Gates amp are closed at the StateFairOfTX
5749,2019-10-12 17:31:19+00:00,DallasPD,All gates at the StateFairOfTX are closed Only credential parking allowed in TexasFootball OUFootball RRShowdown BigTex ChiefHallDPD
5761,2019-10-11 19:30:56+00:00,DallasPD,Gate at the StateFairOfTX is closed Parking lot is full BigTex


In [449]:
df_1.shape

(14074, 3)

In [454]:
mask2_1 = df_1['tweet'].str.contains('parking', regex = False, case = False, na=False)
mask2_2 = df_1['tweet'].str.contains('gate', regex = False, case = False, na=False)
mask2_3 = df_1['tweet'].str.contains('parade', regex = False, case = False, na=False)
mask2_4 = df_1['tweet'].str.contains('amp', regex=False, case=False, na=False)
mask2_5 = df_1['tweet'].str.contains('StateFairOfTX', regex=False, case=False, na=False)
mask2_6 = df_1['tweet'].str.contains('open', regex=False, case=False, na=False)
mask2_7 = df_1['tweet'].str.contains('school', regex=False, case=False, na=False)
mask2_8 = df_1['tweet'].str.contains('fire', regex=False, case=False, na=False)
mask2_9 = df_1['tweet'].str.contains('ALERT', regex=False, case=False, na=False)
mask2_10 = df_1['tweet'].str.contains('accident', regex=False, case=False, na=False)
mask2_11 = df_1['tweet'].str.contains('UPDATE', regex=False, case=False, na=False)

layer2 = (mask2_1 | mask2_2 | mask2_3 | mask2_4 
          | mask2_5| mask2_6|mask2_7|mask2_8|mask2_9
          |mask2_10|mask2_11)

In [455]:
df_1 = df_1[~layer2]

In [456]:
df_1.shape

(8538, 3)

In [458]:
df_1.head(100)

,dates&time,user,tweet
4959,2020-02-06 22:03:12+00:00,DallasPD,DallasPD and dfrincidents are currently on location in the block of Main St regarding a suspicious package Roads are closed for safety precation at HarwoodMain St PearlCommerce PearlMain ChiefHallDPD
5983,2019-08-30 21:16:20+00:00,DallasPD,Monday Sept on LaborDay Jack Evans Police Hdqtrs will be closed for business on Sept in observance of LaborDayCesar E Chavez Holiday For specific details please go to For nonemergency calls call For emergency calls call
6062,2019-08-14 22:31:39+00:00,DallasPD,PIODPD is at the scene of a possible barricaded person in the block of Hillburn Dr Hillburn between Bruton Rd and Tillman St is closed in both directions Bruton Rd between Alhambra St and Gaylord Dr is also closed in both directions
6175,2019-07-13 22:04:27+00:00,DallasPD,PIODPD is on scene of an Officer Involved Shooting in the block of Great Trinity Forest Way Both roads of travel on Great Trinity Forest Way are closed between I and Haas Drive
6177,2019-07-13 01:38:19+00:00,DallasPD,Major police incident in downtown Dallas Griffin Street between Main Street and Pacific is closed in both directions
6364,2019-06-09 21:42:32+00:00,DallasPD,Due to the Crane Collapse on Live Oak the following streets are closed Good Latimer at Gaston Good Latimer at Live Oak Live Oak at Cantegral and Live Oak at Texas The Family Unification Center is located at The Latino Cultural Center located at Live Oak
6930,2019-02-23 02:33:30+00:00,DallasPD,On the scene of officer involved shooting No injuries reported Fair Oaks Ave to Phoenix St closed More info will be released when it becomes available
7806,2018-08-17 11:56:11+00:00,DallasPD,Street closure both Northbound and Southbound N Jim Miller between Military Parkway and Parkdale Drive are closed at this time
8137,2018-05-31 20:14:19+00:00,DallasPD,Eastbound Commerce from Griffin St is closed and southbound Field St from Main St is closed and south bound Arkard Street from Main St
8232,2018-05-05 14:14:51+00:00,DallasPD,Saturday May from am to pm Young Street between Ervay Street and Griffin Street will be closed and South Akard Street between Canton Street and Young Street will also be closed during the same time period Additional roads may be closed as needed


In [460]:
# Second dataframe

df_2 = df1[~layer_1]

In [462]:
df_2.head()

,dates&time,user,tweet
0,2016-10-30 20:51:45+00:00,DentonPolice,Investigating mismatch of name photo Until then view the source directly from City Jail Custody Report page
1,2016-10-30 16:59:03+00:00,DentonPolice,Regina yrs old Bond DRIVING WHILE INTOXICATED RD OR MORE
2,2016-10-30 13:52:19+00:00,DentonPolice,Steven yrs old Bond SPEEDING MPH IN A MPH ZONE DL DRIVING WHILE LICENSE INVALI
3,2016-10-26 21:45:57+00:00,DentonPolice,Monique yrs old Bond POSS MARIJ lt OZ
4,2016-10-26 21:09:45+00:00,DentonPolice,Katelyn yrs old Bond PUBLIC INTOXICATIONNONALCOHOL


In [463]:
df_2.shape

(159903, 3)

In [ ]:
#df_1.duplicated().sum()

In [ ]:
#df_1.drop_duplicates(keep='first', inplace = True)

In [464]:
df_1['category '] = 1

In [465]:
df_2['category '] = 0

/Users/songmei/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [467]:
df_1.head(2)

,dates&time,user,tweet,category
4959,2020-02-06 22:03:12+00:00,DallasPD,DallasPD and dfrincidents are currently on location in the block of Main St regarding a suspicious package Roads are closed for safety precation at HarwoodMain St PearlCommerce PearlMain ChiefHallDPD,1
5983,2019-08-30 21:16:20+00:00,DallasPD,Monday Sept on LaborDay Jack Evans Police Hdqtrs will be closed for business on Sept in observance of LaborDayCesar E Chavez Holiday For specific details please go to For nonemergency calls call For emergency calls call,1


In [468]:
df_2.head(2)

,dates&time,user,tweet,category
0,2016-10-30 20:51:45+00:00,DentonPolice,Investigating mismatch of name photo Until then view the source directly from City Jail Custody Report page,0
1,2016-10-30 16:59:03+00:00,DentonPolice,Regina yrs old Bond DRIVING WHILE INTOXICATED RD OR MORE,0


### Populate final dataframe.

In [469]:
df_final = pd.concat([df_1, df_2])

In [470]:
df_final['category '].value_counts()

0    159903
1    8538  
Name: category , dtype: int64

In [474]:
df_final.to_csv('Final_data.csv')